In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import tensorflow_probability as tfp
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import seaborn as sns
sns.reset_defaults()
import sys
tfd = tfp.distributions
sns.set_context(context='talk',font_scale=0.7)
%matplotlib inline
from model import ModelFunc
from dataset import *
from scipy.io import loadmat
import h5py
from h5py import File
import pickle as pickle
from scipy.interpolate import interp1d

In [ ]:
# hyperparameter configurations

BATCH_SIZE = 32
EPOCHS = 10
MODEL_NAME  = "TEST_ESTIMATOR"
SAVEFOLDER = "model_for_tutorial_test" # when doing demo, use the model named "model_for_tutorial" for testing and evaluation do not train
TRAIN_DATA_PATH = "final_whitened_data_with_labels_combined.h5"
TEST_DATA_PATH = "final_whitened_data_with_labels_combined.h5"
MIN_SNR = 1.


try:
    os.mkdir(SAVEFOLDER)
except:
    os.system('rm -rf ' + SAVEFOLDER)
    os.mkdir(SAVEFOLDER)


config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

custom_config = tf.estimator.RunConfig(save_summary_steps = 100,
                                       log_step_count_steps = 100,
                                       save_checkpoints_steps=300,
                                       session_config=config
                                       )

predictor = tf.estimator.Estimator(model_fn = ModelFunc,
                                   config = custom_config,
                                   model_dir = SAVEFOLDER)

tensors_to_log = {"relative_error_test_out": 'relative_error_test',
                  "loss_out": 'loss'}

logging_hook = tf.compat.v1.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)


In [ ]:
# import data and visualize
with h5py.File('final_whitened_data_with_labels_combined.h5', 'r') as f:
    data_x = f[u'train_data'][:10]
    labels = f[u'train_label'][:10]
plt.plot(data_x[9])
#plt.title('visualization')

# Training & Evaluation

In [ ]:
# training

for i in range(EPOCHS):
    predictor.train(input_fn=lambda: generator(filename=TRAIN_DATA_PATH, shuffle=False,
                    batch_size = BATCH_SIZE, min_SNR=MIN_SNR))
    metrics = predictor.evaluate(input_fn=lambda: generator(filename=TEST_DATA_PATH, shuffle=False,
                                 batch_size = BATCH_SIZE, min_SNR=MIN_SNR))
    print('#'*30)
    print('metrics {} at epoch {}'.format(metrics, i+1))
    print('#'*30)

print('Finished!')

# Reload Model into Estimator

In [ ]:
#SAVEFOLDER = "model_for_tutorial_2"
config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
custom_config = tf.estimator.RunConfig(save_summary_steps = 100,
                                       log_step_count_steps = 100,
                                       save_checkpoints_steps=300,
                                       session_config=config
                                       )

predictor = tf.estimator.Estimator(model_fn = ModelFunc,
                                   config = custom_config,
                                   model_dir = SAVEFOLDER)

# Evaluation on different SNRs

In [ ]:
# evaluation on test set
check_SNR = 1.
TEST_DATA_PATH = "final_whitened_data_with_labels_combined.h5"
BATCH_SIZE = 32
metrics_pred = predictor.evaluate(input_fn=lambda: tester_fixed_SNR(filename=TEST_DATA_PATH, shuffle=False,
                                 batch_size = BATCH_SIZE, SNR=check_SNR))

print(metrics_pred)

# Make predictions on different SNRs

In [ ]:
# prediction on test set (inference)
check_SNR = 3.
TEST_DATA_PATH = "final_whitened_data_with_labels_combined.h5"
BATCH_SIZE = 32
data_predict = predictor.predict(input_fn=lambda: tester_fixed_SNR(filename=TEST_DATA_PATH, shuffle=False,
                                 batch_size = BATCH_SIZE, SNR=check_SNR))


one_batch_prediction = data_predict.next()
print(np.std(one_batch_prediction['predictions'])) 

In [ ]:
_ = plt.hist(one_batch_prediction['predictions'].squeeze(), bins=30)

# Save model for prediction (another approach)
The above approach requires loading models for every evaluation, which is slow. Here I'll introduce a new way to make inferences after the model is trained.

In [ ]:
def serving_input_receiver_fn():
    """Serving input_fn that builds features from placeholders

    Returns
    -------
    tf.estimator.export.ServingInputReceiver
    """
    data = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 8192], name='data')
    receiver_tensors = {'data': data}
    return tf.estimator.export.ServingInputReceiver(data, receiver_tensors)

In [ ]:
predictor.export_saved_model('saved_model_test', serving_input_receiver_fn)